In [92]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
from scipy import signal as sig
import wfdb
from wfdb import processing as wfdbp
import pandas as pd

In [94]:
#ECG Denoising
fs = 360

def pli_filtering(signal, fs, Q = 20, pl_freq = 60): #PLI filtering (60 Hz), Quality factor: 20
    wc = pl_freq / (fs / 2)
    b, a = sig.iirnotch(wc, Q)
    return sig.filtfilt(b, a, signal)

def bw_filtering(signal, fs, N = 2, bw_freq = 0.5): #Baseline Wander filtering (0.5 Hz), Butterworth order: 2
    wc = bw_freq / (fs / 2)
    b, a = sig.butter(N, wc, 'high')
    return sig.filtfilt(b, a, signal)
    
def lp_filtering(signal, fs, N = 2, fc = 150): #LP filtering (150 Hz), Butterworth order: 2
    wc = fc / (fs / 2)
    b, a = sig.butter(N, wc, 'low')
    return sig.filtfilt(b, a, signal)


def ecg_denoising(signal, fs):
    signal1 = lp_filtering(signal, fs)
    signal2 = bw_filtering(signal1, fs)
    signal3 = pli_filtering(signal2, fs)
    return signal3

In [96]:
#ECG Segmentation

def get_sig_ann_fromId(rec_id):
    
    path = 'mit-bih-arrhythmia-database-1.0.0'
    record = wfdb.rdrecord(path + '/' + rec_id)
    ann = wfdb.rdann(path + '/' + rec_id, 'atr')

    # All annotation sample indices:
    all_ann_samples = np.asarray(ann.sample)
    all_ann_symbols = np.asarray(ann.symbol)

    BEAT_SYMBOLS = {
        # Normal / bundle branch blocks:
        'N','L','R','e','j',
        # Supraventricular:
        'A','a','J','S',
        # Ventricular:
        'V','E',
        # Fusion:
        'F',
        # Other:
        '/','Q', 'f'
    }

    is_beat = np.isin(all_ann_symbols, list(BEAT_SYMBOLS))
    beat_indices = all_ann_samples[is_beat]    
    beat_symbols = all_ann_symbols[is_beat]  
    signal = record.p_signal[:,0] #Single lead
    return signal, ann


def simplify_types(annotations): #I will simplify to Normal, Supraventricular, Ventricular and Fusion beats, classifying will be performed with arrythmic (Supra, Ventri and Fusion) / non-arryhmic (Normal)

    ann_samples = np.asarray(annotations.sample)
    ann_symbols = np.asarray(annotations.symbol)
    
    for i in range(len(ann_samples)): #Could optimize this for sure
        if ann_symbols[i] in ['N', 'L', 'R', 'e', 'j']: #Normal
            ann_symbols[i] = 'N'
        elif ann_symbols[i] in ['A', 'a', 'J', 'S']: #Supraventricular
            ann_symbols[i] = 'S'
        elif ann_symbols[i] in['V', 'E']: #Ventricular
            ann_symbols[i] = 'V'
        elif ann_symbols[i] in ['F']: #Fusion
            ann_symbols[i] = 'F'
        elif ann_symbols[i] in ['/', 'Q', 'f']:
            ann_symbols[i] = 'Unk'

    good_beat = np.isin(ann_symbols, ['N', 'S', 'V', 'F', 'Unk']) 
    beat_indices = ann_samples[good_beat]    
    beat_symbols = ann_symbols[good_beat]  

    return beat_indices, beat_symbols

def get_segments(record, annotations, fs = 360, n = 5, ins_num = 1000, min_beats = 1):
    #n = Number of beats contained within each instance
    beat_ranges = [int(-0.2 * fs), int(0.4 * fs)]#beat_ranges: For now fixed (-0.2s from first beat to +0.4s from last beat)
    
    beat_indices, beat_symbols = simplify_types(annotations)
    segments = [] #Beat signal associated with each instance, won´t be stored in attribute array, but will have same index as ID
    beat_types = [] #Same as segments, will not a direct attribute, but will be stored with the same ID (Stored in ASCII value)
    rel_indices = [] #Relative peak indices for each segment
    
    max_segments = max(0, len(beat_indices) - (n - 1))
    seg_num = min(ins_num, max_segments)
    instances = np.full((seg_num, 10), np.nan, dtype=float)
    
    for i in range(seg_num):
        types = [] #Sublist for beat_types within each segment
        indices = [] #Sublist for rel_indices within each segment
        start = max(beat_indices[i] + beat_ranges[0], 0)
        end = min(beat_indices[i + n - 1] + beat_ranges[1], len(record))
        segments.append(record[start : end])

        isArrythmic = 0.0
        arr_beat_counter = 0
        for peak in range(i, i + n):
            types.append(beat_symbols[peak]) 
            indices.append(beat_indices[peak] - start)
            if beat_symbols[peak] not in ['N', 'Unk']:
                arr_beat_counter += 1
        if arr_beat_counter >= min_beats:
            isArrythmic = 1.0
        beat_types.append(types)
        rel_indices.append(indices)

        instances[i, 0] = i
        instances[i, 1] = isArrythmic
        
    return instances, segments, beat_types, rel_indices

In [98]:
#Feature Selection (Methods)

#def compute_mean(segment):
#    return np.mean(segment)

def compute_rmssd(indices, fs): #Computed through indices, not Pan-Tomkins
    return np.sqrt(np.mean(np.diff(indices) ** 2)) / fs 

def compute_std_rr(indices, fs): #Computes std between beat R-R intervals
    return np.std(np.diff(indices) / fs)

def compute_rms(segment):
    return np.sqrt(np.mean(segment ** 2))

def compute_mean_qrs_length(on, off, fs):
    return np.mean(off - on) / fs

def compute_relative_qrs_energy(segment, on, off): #Computes the ratio between the QRS segments energy and the total energy 
    total_energy = np.sum(segment ** 2)
    qrs_energy = 0
    for i in range(len(on)):
        for j in range(on[i], off[i]):
            qrs_energy += segment[j] ** 2
    return qrs_energy / total_energy

def compute_mean_peak_amp(segment, indices):
    return np.mean(abs(segment[indices] - np.median(segment)))
    
def compute_psd(segment, fs): #PSD computed through Welch´s (in Hz)
    return sig.welch(segment - np.mean(segment), fs)

def compute_relative_power(f, Pxx, bw): #Computes the ratio between the power of the specified band and the whole psd
    total_power = np.trapz(Pxx, f)
    idx = (f >= bw[0]) & (f <= bw[1])
    qrs_power = np.trapz(Pxx[idx], f[idx])
    return qrs_power / total_power
#QRS Complex: (5 - 15Hz), T Wave: (0.5 - 10 Hz), P Wave: (5 - 30 Hz)
    
def compute_power_percentile(f, Pxx, bw = [0.5, 40], percentile = 0.95): #Computes within the specified bw in which frequency value the cumulative power reached the specified percentile
    idx = (f >= bw[0]) & (f <= bw[1])
    f_band = f[idx]
    P_band = Pxx[idx]

    df = np.diff(f_band)
    df = np.r_[df, df[-1]]

    cum_power = np.cumsum(P_band * df)
    cum_power /= cum_power[-1]

    return f_band[np.searchsorted(cum_power, percentile)]
#I will try using a .95 percentile for the bw that contains the most relevant ECG freq. components (0.5 - 40 Hz)

In [100]:
#Feature Selection
#Features: 0.ID, 1.Arrythmic? (Contains 1+ Arrythmic beat) [0: True, 1:False], 2.RMSSD, 3.Std_RR, 4.RMS_ECG, 
#5.Mean QRS Length, 6.QRS Energy ratio, 7.Mean R peak amp, 8.QRS band Power ratio, 9. Frequency at 0.95 power within (0.5 - 40 Hz)

def feature_selection(instances, segments, indices, fs = 360):
    for i in range(len(instances)):
        #Time features
        instances[i, 2] = compute_rmssd(indices[i], fs) #RMSSD (Root mean squared successive diffs),
        instances[i, 3] = compute_std_rr(indices[i], fs) #3.Std between beats (R-R interval)
        instances[i, 4] = compute_rms(segments[i]) #4.Rms of ECG amplitude
        on, off, act = qrs_onset_offset(segments[i], indices[i], fs)
        instances[i, 5] = compute_mean_qrs_length(on, off, fs) #5.Mean QRS Length (Duration)
        instances[i, 6] = compute_relative_qrs_energy(segments[i], on, off) #6.QRS Energy Ratio
        instances[i, 7] = compute_mean_peak_amp(segments[i], indices[i]) #7. Mean R peak amp
        
        #Freq. features
        f, Pxx = compute_psd(segments[i], fs)
        instances[i, 8] = compute_relative_power(f, Pxx, [5, 15]) #8. QRS band Power Ratio
        instances[i, 9] = compute_power_percentile(f, Pxx, [0.5, 40], 0.95) #Frequency at 0.95 power within [0.5 - 40 Hz]      
        
    return instances

In [102]:
#Detect QRS complex onset and offset (for now I will use Pan-Tomkins)

def pass_band_filtering(segment, fs = 360, low = 5, high = 15, N = 2): # 5 - 15 Hz (from the paper)
    low_c = low / (fs/2)
    high_c = high / (fs/2)
    b,a = sig.butter(N, [low_c, high_c], 'bandpass')
    return sig.filtfilt(b, a, segment)

def sav_golay(segment, fs = 360, polyorder = 3, deriv = 1, window = 13): #Savinsky-Golay differentiator
    return sig.savgol_filter(segment, window, polyorder, deriv, (1/fs), mode = 'interp')

def ma_integration(segment, fs = 360, window = int(0.05 * fs)): #150 ms (from the paper), 50 ms for clearer QRS isolation
    b = np.ones(window) / window
    return sig.filtfilt(b, 1, segment)

#1.Bandpass filtering --> 2.Differentiation --> 3.Squaring --> 4.MA Itegration
def pan_tomkins(segment, fs = 360):
    segment_f = pass_band_filtering(segment, fs)
    segment_d = sav_golay(segment_f, fs)
    segment_s = segment_d ** 2
    segment_i = ma_integration(segment_s, fs)
    return segment_i

#Compute the onset & offset
def qrs_onset_offset(segment, indices, fs, back_ms = 180, fwd_ms = 20, base_ms = (-220, -140), alpha = 0.25, hold_ms = 30):
    
    #alpha: Threshold (activity peak fraction)
    #hold_ms: It must remain below hold_ms to confirm a return to baseline
    
    act = pan_tomkins(segment, fs)

    back = int(back_ms * fs / 1000)
    fwd  = int(fwd_ms  * fs / 1000)
    hold = max(1, int(hold_ms * fs / 1000))

    on  = np.full(len(indices), -1, dtype=int)
    off = np.full(len(indices), -1, dtype=int)

    for i, r in enumerate(indices):
        lo = max(r - back, 0)
        hi = min(r + fwd, len(act) - 1)

        #base: baseline
        b0 = max(r + int(base_ms[0] * fs / 1000), 0)
        b1 = min(r + int(base_ms[1] * fs / 1000), len(act))
        base = np.median(act[b0:b1]) if b1 > b0 else np.median(act[lo:r])

        peak = np.max(act[lo:hi+1])
        thr = base + alpha * (peak - base)

        # onset: Backwards until below threshold during hold
        onset = lo
        for k in range(r, lo, -1):
            if np.all(act[max(k - hold, 0) : k] < thr):
                onset = k
                break

        # offset: Forwards until below threshold during hold
        offset = hi
        for k in range(r, hi-hold):
            if np.all(act[k : k + hold] < thr):
                offset = k
                break

        on[i] = onset
        off[i] = offset

    return on, off, act

In [104]:
#Save features in CSV
LABEL_COLUMN = "Arrhythmic"

FEATURE_COLUMNS = [
    "RMSSD",
    "Std_RR",
    "RMS_ECG",
    "Mean_QRS_Length",
    "QRS_Energy_Ratio",
    "Mean_R_Peak_Amp",
    "QRS_Band_Power_Ratio",
    "Freq_95pct_Power",
]

def save_features_sklearn(instances, filename, rec_id):
    instances = np.asarray(instances)

    df = pd.DataFrame({
        "RecordID": rec_id,  # pandas lo replicará a todas las filas
        LABEL_COLUMN: instances[:, 1].astype(int),
        "RMSSD": instances[:, 2],
        "Std_RR": instances[:, 3],
        "RMS_ECG": instances[:, 4],
        "Mean_QRS_Length": instances[:, 5],
        "QRS_Energy_Ratio": instances[:, 6],
        "Mean_R_Peak_Amp": instances[:, 7],
        "QRS_Band_Power_Ratio": instances[:, 8],
        "Freq_95pct_Power": instances[:, 9],
    })

    df_sklearn = df[["RecordID", LABEL_COLUMN] + FEATURE_COLUMNS]
    df_sklearn.to_csv(filename, index=False)
    return df_sklearn


In [106]:
rec_id = []

for i in range(25):
    if (i == 10) | (i == 20) | (i == 2) | (i == 4) | (i == 7):
        continue
    rec_id.append(str(100 + i))
    
for i in range(35):
    if(i == 4) | (i == 6) | (i == 11) | (i == 16) | (i == 18) | (i >= 24 | i <= 27) | (i == 29) | (i == 17):
        continue
    rec_id.append(str(200 + i))

                             
print(rec_id)

['100', '101', '103', '105', '106', '108', '109', '111', '112', '113', '114', '115', '116', '117', '118', '119', '121', '122', '123', '124', '200', '201', '202', '203', '205', '207', '208', '209', '210', '212', '213', '214', '215', '219', '220', '221', '222', '223', '228', '230', '231', '232', '233', '234']


In [110]:
fs = 360

for n in [3, 5, 7]:
    all_rows = []
    for i in range(len(rec_id)):
        signal, ann = get_sig_ann_fromId(rec_id[i])
        denoised_signal = ecg_denoising(signal, fs)
        instances, segments, beat_types, indices = get_segments(denoised_signal, ann, n = n, min_beats = 1)
        instances = feature_selection(instances, segments, indices, fs)
        df = save_features_sklearn(instances, 'Data_final/' + str(n) + '_n_' + rec_id[i] + '.csv', rec_id[i])
        all_rows.append(df)

    final_df = pd.concat(all_rows, ignore_index = True)
    final_df.to_csv('Data_final/' + str(n) + '_n_all_records.csv', index = False)

